In [1]:
import pandas as pd
from skimage.external import tifffile
import os
os.sys.path
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import numpy as np

In [2]:
bigDir = r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\DATA'

foldersToLook = ['20190506-MSL decon rename', '20190607-MSL decon rename', '20190609-MSL decon rename', 
                 '20190625-MSL decon rename', '20190701-MSL decon rename', '20190712-MSL decon rename',
                 '20190716-MSL decon rename']

## by 10%

In [3]:
for i in range(len(foldersToLook)):
    currFolder = foldersToLook[i]
    print(currFolder)
    currBigDir = f'{bigDir}\\{currFolder}'

    currMaskFolder = f'{currBigDir}\\segmentation'
    currCellFolder = f'{currBigDir}\\data_tiff'
    os.mkdir(f'{currBigDir}\\segmentation_10')
    newSegFolder = f'{currBigDir}\\segmentation_10'

    segFiles = []
    cellFiles = []

    for (dirpath, dirnames, filenames) in os.walk(currMaskFolder):
        segFiles.extend(filenames)
        break
    for (dirpath, dirnames, filenames) in os.walk(currCellFolder):
        cellFiles.extend(filenames)
        break

    nucleusMasks = [f for f in segFiles if "nucleus" in str(f)]

    
    if 'Thumbs.db' in nucleusMasks:
        nucleusMasks.remove('Thumbs.db')
    elif 'Thumbs.db' in cellFiles:
        cellFiles.remove('Thumbs.db')

    
    for j in range(len(hetFiles)):
        
        myImage = tifffile.imread(f'{currCellFolder}\\{cellFiles[j]}')
        myChannel = myImage[:, 1, :, :]
        myMask = tifffile.imread(f'{currMaskFolder}\\{nucleusMasks[j]}')
        myMask = myMask.astype(bool)

        myHetMask = myChannel[myMask]

        myPercentile = np.nanpercentile(myHetMask, 90) 


        myChannel[myChannel < myPercentile] = 0
        myChannel[myChannel >= myPercentile] = 1
        myChannel[~myMask]=0
        myFile = cellFiles[j].split(".")[0]

        tifffile.imsave(f'{newSegFolder}\\{myFile}.tiff', myChannel)
    

20190506-MSL decon rename


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'Z:\\CookLab\\Liu\\20190816_organizedData_MCM_loading\\DATA\\20190506-MSL decon rename\\segmentation_10'

In [35]:
for i in range(len(foldersToLook)):
    currFolder = foldersToLook[i]
    currBigDir = f'{bigDir}\\{currFolder}'
    
    folder = f'{currBigDir}\\segmentation_10'
    
    for filename in os.listdir(folder):
        filename_changed = filename.split(".")[0] 
        changed = filename_changed + '_hetChrom_10'
        
        src = f'{folder}\\{filename}'
        dst = f'{folder}\\{changed}.tiff'
        os.rename(src, dst)
        
    

## data collection for 10%

In [4]:
dataFrame10 = pd.DataFrame(columns = ['File', 'Ch1_HetChrom', 'Ch1_Euchrom', 'Ch1_Total','Ch2_HetChrom', 'Ch2_Euchrom', 'Ch2_Total','Ch3_HetChrom', 'Ch3_Euchrom', 'Ch3_Total','nucleusVolume', 'hetVolume'])

In [5]:
dataFrame10

,File,Ch1_HetChrom,Ch1_Euchrom,Ch1_Total,Ch2_HetChrom,Ch2_Euchrom,Ch2_Total,Ch3_HetChrom,Ch3_Euchrom,Ch3_Total,nucleusVolume,hetVolume


In [12]:
def setinDF (folderName):
    currDir = f'{bigDir}\\{folderName}'
    currOrgCellFolder = f'{currDir}\\data'
    currCellFolder = f'{currDir}\\data_tiff'
    currNucFolder = f'{currDir}\\segmentation'
    currHetFolder = f'{currDir}\\segmentation_10'
    
    nucFiles = []
    cellFiles = []
    hetFiles = []
    orgFiles = []

    for (dirpath, dirnames, filenames) in os.walk(currOrgCellFolder):
        orgFiles.extend(filenames)
        break
    for (dirpath, dirnames, filenames) in os.walk(currNucFolder):
        nucFiles.extend(filenames)
        break
    for (dirpath, dirnames, filenames) in os.walk(currCellFolder):
        cellFiles.extend(filenames)
        break
    for (dirpath, dirnames, filenames) in os.walk(currHetFolder):
        hetFiles.extend(filenames)
        break
        
    nucleusMasks = [f for f in nucFiles if "nucleus" in str(f)]
    
    if 'Thumbs.db' in hetFiles:
        hetFiles.remove('Thumbs.db')
    elif 'Thumbs.db' in cellFiles:
        cellFiles.remove('Thumbs.db')
    elif 'Thumbs.db' in orgFiles:
        orgFiles.remove('Thumbs.db')
    
    df = pd.DataFrame(columns =  ['File', 'Ch1_HetChrom', 'Ch1_Euchrom', 'Ch1_Total','Ch2_HetChrom', 'Ch2_Euchrom', 'Ch2_Total','Ch3_HetChrom', 'Ch3_Euchrom', 'Ch3_Total','nucleusVolume', 'hetVolume'])
    
    for i, file in enumerate(orgFiles):
        df.loc[i, "File"] = file
    print(len(orgFiles))
    print(len(cellFiles))
    print(len(hetFiles))
    print(len(nucleusMasks))
    for i in range(len(orgFiles)):
        channel1 = extractData(currDir, cellFiles[i], hetFiles[i], nucleusMasks[i], 0)
        channel2 = extractData(currDir, cellFiles[i], hetFiles[i], nucleusMasks[i], 1)
        channel3 = extractData(currDir, cellFiles[i], hetFiles[i], nucleusMasks[i], 2)
        
        df.loc[i, 'Ch1_Total'] = channel1[0]
        df.loc[i, 'Ch1_HetChrom'] = channel1[1]
        df.loc[i, 'Ch1_Euchrom'] = channel1[2]
        
        df.loc[i, 'Ch2_Total'] = channel2[0]
        df.loc[i, 'Ch2_HetChrom'] = channel2[1]
        df.loc[i, 'Ch2_Euchrom'] = channel2[2]
        
        df.loc[i, 'Ch3_Total'] = channel3[0]
        df.loc[i, 'Ch3_HetChrom'] = channel3[1]
        df.loc[i, 'Ch3_Euchrom'] = channel3[2]
        
        df.loc[i, 'nucleusVolume'] = channel1[3]
        df.loc[i, 'hetVolume'] = channel1[4]
        
    
    return df

In [7]:
def extractData (currDir, cell, het, nuc, channel):
    myImage = tifffile.imread(f'{currDir}\\data_tiff\\{cell}')

    myChannel = myImage[:, channel, :, :]

    myNucleusMask = tifffile.imread(f'{currDir}\\segmentation\\{nuc}')
    myNucleusMask = myNucleusMask.astype(bool)

    myHeteroMask = tifffile.imread(f'{currDir}\\segmentation_10\\{het}')
    myHeteroMask = myHeteroMask.astype(bool)
    
    myTotal = np.sum(myChannel[myNucleusMask])
    myHet = np.sum(myChannel[myHeteroMask])
    
    myTotalVolume = np.sum(myChannel[myNucleusMask])
    myHetVolume = np.sum(myChannel[myHeteroMask])
    
    myEuchro = myTotal - myHet

    result = []
    result.append(myTotal)
    result.append(myHet)
    result.append(myEuchro)
    result.append(myTotalVolume)
    result.append(myHetVolume)
    
    
    return result

In [13]:
#for i in range(len(foldersToLook)):
print(foldersToLook[0])
df_set =  setinDF(foldersToLook[0])
dataFrame10 = dataFrame10.merge(df_set, how = 'outer', on =  ['File', 'Ch1_HetChrom', 'Ch1_Euchrom', 'Ch1_Total','Ch2_HetChrom', 'Ch2_Euchrom', 'Ch2_Total','Ch3_HetChrom', 'Ch3_Euchrom', 'Ch3_Total','nucleusVolume', 'hetVolume'])

20190506-MSL decon rename
18
18
29
0


IndexError: list index out of range

In [ ]:
dataFrame10